In [1]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_7JLSus_U3m4SxY6snjBuCB5KAqBXGMm2h5YrZkSicYdCqQhVwDCNVGrybyf26H8MQVDwTa")

In [2]:
index_name = "document-store"

In [3]:
pinecone_index = pc.Index(index_name)

In [11]:
# Create index if not exists
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Match Bedrock embedding dimensions
        metric="cosine",
        spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
    )
pinecone_index = pc.Index(index_name)

In [4]:
import streamlit as st
import random
import time
import pandas as pd
import os
from urllib.parse import parse_qs
import boto3
from langchain.llms.bedrock import Bedrock
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import json
from dotenv import load_dotenv

load_dotenv()

session = boto3.Session(profile_name="personal", region_name="us-east-1")
bedrock = boto3.client("bedrock-runtime")

In [5]:
import json
from langchain_text_splitters import RecursiveCharacterTextSplitter

def process_text_file(file_path, username):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.create_documents([text])
    
    embeddings = []
    for chunk in chunks:
        body = json.dumps({"inputText": chunk.page_content})
        
        response = bedrock.invoke_model(
            body=body,
            modelId="amazon.titan-embed-text-v2:0",
            accept="application/json"
        )
        
        # Fixed response parsing
        response_body = json.loads(response['body'].read())
        if 'embedding' in response_body:
            embeddings.append(response_body['embedding'])
    
    vectors = [{
        "id": str(hash(chunk.page_content)),
        "values": embedding,
        "metadata": {
            "user_id": username,
            "text": chunk.page_content,
            "source": file_path
        }
    } for chunk, embedding in zip(chunks, embeddings)]
    
    return vectors  # Now returns list of vectors

    # pinecone_index.upsert(vectors)


In [6]:
vectors = process_text_file("linkedin_posts (2).txt", username="sunjana")

In [7]:
pinecone_index.upsert(vectors)

{'upserted_count': 79}